# Virtual Memory

## Background

Programs don't need to be loaded in to memory in its entirety. We can run sections of a program at a time, and this increases CPU utilization with no increase in response. When there is less I\O needed to load or swap programs into memory each user program runs faster.

### Virtual memory

Virtual Memory - Separation of user logical/virtual memory from physical memory

Because only part of the program needs to be in memory for execution at any point in time
- The logical address space can be larger than the physical address space
- Allows address spaces to be shared by several processes
- Allows for more efficient process creation
- More programs running concurrently
- Less I\O needed to load or swap processes

Virtual Address Space - the logical view of how process is stored in memory
- usually starts at address 0, and is continuous until the end of memory needed
- meanwhile, the physical memory is organized in page frames
- MMU must map the logical space to the virtual space

Virtual Memory can be implemented via __demand paging__ and __demand segmentation__


<div style="text-align: center"><img src="Pictures/virtual_memory.png"></div>

Virtual address space is the address space each program is allowed to reference, and the cpu will turn into physical address.

Top to bottom we have a 
Stack - Stores stack pointers, used for function calls and recursion, grows downwards
Heap - Stores dynamically allocated memory blocks, grows upwards
Data - Stores the static Data
Code - Stores the Code. Sometimes called the text section

THis stack and heap allow for sparse address spaces with holes left for growth. __System Libraries are shared via mapping__. Pages can be shared during fork, speeding creation



## Demand Paging

Could bring entire process into at load time, or it can bring a page into real memory only when it is needed.

When we ony bring the page into
- less io is needed, no unnecessary io
- less memory is needed
- Faster response
- More Users

This is like a paging system, but with swapping, and more granular

If a page is needed, then reference it
- if it is a valid reference, abort
- if it is valid but not in memory, then bring it to memory

This is a lazy swapper, which means it never swaps a page into memory unless the page is needed
- a swapper that deals with pages is called a pager


<div style="text-align: center"><img src="Pictures/demand_paging.png"></div>

## Swapping

### Basic Concepts

With swapping the pager guesses how many pages will be used before paging out again

Instead, the pager only brings in needed pages
- hardware needs to be fast
- We need an MMU functionality to implement demand paging

If pages needed are already memory resident
- then there is no difference between non demand paging

Accessing memory from ram is slow, this is why hyperthreading is good (more cpu registers). Accessing the disk is even slower, so we need to keep the cpu busy

Each Process has its own page table. The page table indicates if the page is in RAM or disk

With each page table entry, there is a valid and invalid bit associated with each page

V means page is memory resident and i means not in memory

Durring MMU Address translation, if the valid invalid bit is i, then we get a page fault


# Page Replacement

Paging Out = moving from main memory to secaondary memory

Frame Table = keeps track of what blocks of physical memory are mapped to what page

If the page is not in the page Table, then we find the desired page from secondary memory.

Find a new free frame in the processes page table, hopefully one that is not in use.
- If there is a free frame
    - use that frame to store the page {id/data}
- else use a lage replacement algo to select a victim frame
    - write the victim frame to secondary storage if needed
    - change the page and freme table   
    - continue the process form where the page fault occured

## Modify And Dirty Bit

Used to increase performance Each Page or frame has a modify/dirty Bit

This is set when a byte in the page is writteninto indicating that the page has been modified. Page Data != Physical Memory Data

If the dirty bit is set, then we must write the changes of the page to the memory and then we can access the page

If not then we can directly access the page



## Page Fault

If there is a reference to a page to read or write, the first reference to that page will trap the operating system (since it is not yeat in the page table and the valid bit is 'i')

1. The operating system looks at another table to see If
- invalid reference -> abort
- Just not in main memory

2. Find a Free Frame in the ram

3. Move the Page from Disk Frame into the RAM frame via a scheduled Disk operation

4. Reset The page table to indicate that the page is now in main memory, and set the valid bit = v

5. Restart the instruction that caused a page Fault


# Aspects of Demand Paging

Extreme - Start With No pages in memory
- OS sets an instruction pointer to first instruction of process, non memory resident which cause a page fault

- __Pure Demand Paging__


- An instruction could access mupltiple pages ona  single command, causing musplt page faults
    -Consider an instruction which adds two numbers and stores the value back into memory. If the two numbers are on different pages we have a problem

    __Locality of REFERENCE__?

We need hardware support for demand Paging
- page table with invalid / valid bit
- secondary memory (swap device with swap space)
- INstruction restart



## Demand Paging Optimization

Swap Space IO is faster than file system IO, even when on the same device.
- Swap is allocated in larger chunks, less management needed than a file system

We copy the entire process image to the swap space at process load time

Page in from program binary on disk, but discard rather than paging out when freeing frame.
- except for data not associated with a file, like stack, heap, and dirty pages



## Copy On Write

vfork and fork system call has a parent suspend and child using copy on write address space on parent.
- designed to have child call exec()

very efficient

Copy On Write allows both parent and child processes to intially share the same pages in memory
- if either process modifies a page then the page is copied

Free pages are allocated from a pool of zero fill on demand pages

The calling process uses the copy of the page in memory because the other process assumes its data has not been changed

## What happens if theres no free frame(No free ram)

Page Replacement- find a page that has not been used for a while and page it out

Prevent over allocation of memory by modifying page fault service to include page Replacement

Use the dirty bit to reduce overhead of page transfers. Only modified pages are written to disk.

# Page and Frame Replacement Algorithms

Frame Allocation algotithm determines
- How many Frames to give each process
- Which frames to Replace

Page Replacement Algorithm
- We want the lowest page fault rate on both first access and reaccess

OPtimize with a reference string

First in first out
- page tables stored in order of page fault. Increment where the next page fault will be stored and keep on moving, cycling through untill we get to the top of the page table. Now we have reaccess page faults where we will page in and page out data

Lest Recently Used
Each page has a counter, look in all pages to see least recently usde

LRU needs special hardware and is still slow.
needs a referenced bit
second chance Algorithm




## Trashing 

If a process doesn't have enough pages, then the fault rate is pretty high 
- low cpu utilization
- os thinking it needs more parallel computing
- another process added further slowing down cpu

Thrashing = busy swapping pages in and out

<div style="text-align: center"><img src="Pictures/thrashing.png"></div>


## Key Topics

Advantages of Paging

How does paging allow virtual memory to work?

Can a process start with no pages in memory, dee demand paging

Note that shared libraries are shared among processes

Why do page tables have a bit that indicates whether a page table is in memory or on disk?

What is the locality of reference and why is it important in paging?

How does copy on write increase performace?

What is the dirty bit and how did it get to be dirty?

How does thrashing work degrade the performance of he computer?

How do you fix thrashing?

What is the translation lookaside buffer?

